- In this Notebook, I am trying to integrate the training data from the backscore and rest of the period from prj-prod-dataplatform.audit_balance.ml_model_run_details table.
- In this I will compare the training period with each month of test period.

**Steps to Follow**:

* Read the specific model data from prj-prod-dataplatform.audit_balance.ml_model_run_details table
* Expand the calcFeature column to extract all the features for the model
* Read the data from specific backscore table for the training data
* Identify the features and create a list
* Use transform_data function to create the same structure as ml_model_run_details table
* Insert the data to a similar training table - prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* Read the specific model data from prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* expand the training set from the calcFeature column
* Concatenate both the test and train datasets
* Calculate the PSI using the PSI function comparing it with the train set
* Insert the result to a PSI table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4

# **PSI - CSI Calculation**

This is based on the decile split of the score and features and for categorical top 20 and rest as others 

## Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

## Function

In [2]:
# Method 1: Using regex to remove all "Calc_" occurrences
def clean_names_regex(name):
    return re.sub(r'_Calc_', '_', name)

# Method 2: Simple string replacement
def clean_names_replace(name):
    return name.replace('_Calc_', '_')

## expand_calc_features_robust

In [3]:
def expand_calc_features_robust(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expand the calcFeatures JSON column into separate columns.
    Column names will be prefixed with modelVersionId_Calc_
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe with 'calcFeatures' column
    
    Returns:
    --------
    pd.DataFrame : Dataframe with expanded features
    """
    df_expanded = df.copy()
    
    if 'calcFeatures' not in df_expanded.columns:
        print("Warning: 'calcFeatures' column not found in dataframe")
        return df_expanded
    
    # Get modelVersionId (assuming all rows have same modelVersionId in filtered dataframe)
    if len(df_expanded) == 0:
        return df_expanded
    
    model_version_id = df_expanded['modelVersionId'].iloc[0]
    prefix = f"{model_version_id}_Calc_"
    
    all_features_data = []
    
    for idx, row in df_expanded.iterrows():
        calc_features_str = row['calcFeatures']
        row_features = {}
        
        if pd.isna(calc_features_str):
            all_features_data.append(row_features)
            continue
            
        try:
            # Clean the string and parse JSON
            features_str = str(calc_features_str)
            # Handle common JSON issues
            features_str = features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            
            features_dict = json.loads(features_str)
            
            if isinstance(features_dict, dict):
                row_features = features_dict
            else:
                print(f"Warning: calcFeatures is not a dictionary at index {idx}")
                
        except json.JSONDecodeError as e:
            print(f"Warning: JSON decode error at index {idx}: {e}")
        except Exception as e:
            print(f"Warning: Error processing calcFeatures at index {idx}: {e}")
        
        all_features_data.append(row_features)
    
    # Create DataFrame from features
    features_df = pd.DataFrame(all_features_data)
    
    # Add prefix to column names
    features_df = features_df.add_prefix(prefix)
    
    # Combine with original dataframe
    df_expanded = df_expanded.reset_index(drop=True)
    features_df = features_df.reset_index(drop=True)
    
    result_df = pd.concat([df_expanded, features_df], axis=1)
    
    # Optionally drop the original calcFeatures column
    if 'calcFeatures' in result_df.columns:
        result_df = result_df.drop('calcFeatures', axis=1)
    
    return result_df

### dropping_duplicates

In [4]:
def dropping_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop duplicates based on digitalLoanAccountId, Data_selection, and modelVersionid,
    keeping the first occurrence based on appln_submit_datetime.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe

    Returns:
    --------
    pd.DataFrame with duplicates dropped
    """

    df = df.sort_values(
        ['digitalLoanAccountId', 'Data_selection', 'modelVersionId', 'appln_submit_datetime'],
        ascending=[True, True, True, True],
        na_position='last'
    )

    result = df.drop_duplicates(
        subset=['digitalLoanAccountId', 'Data_selection', 'modelVersionId'],
        keep='first'
    ).copy()

    return result

### PSI pipeline Version 1

In [5]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional, Union
import warnings
import json
warnings.filterwarnings('ignore')


def calculate_psi_for_model(dfcombined: pd.DataFrame,
                            configdf: pd.DataFrame,
                            model_display_name: str,
                            debug: bool = False) -> pd.DataFrame:
    """
    Calculate PSI for a specific model based on configdf combinations.
    """
    
    # Filter configdf to only include combinations for this specific modelDisplayName
    model_config = configdf[configdf['modelDisplayName'] == model_display_name].copy()
    
    print(f"\n{'='*80}")
    print(f"Starting PSI Pipeline for Model: {model_display_name}")
    print(f"Total combinations to process: {len(model_config)}")
    print(f"{'='*80}\n")
    
    if len(model_config) == 0:
        print(f"ERROR: No configurations found for modelDisplayName={model_display_name}")
        return pd.DataFrame()
    
    # Process each unique combination
    all_results = []
    
    for idx, config_row in model_config.iterrows():
        model_version_id = config_row['modelVersionId']
        trench_category = config_row['trenchCategory']
        
        print(f"Processing combination {idx + 1}/{len(model_config)}: "
              f"modelVersionId={model_version_id}, trenchCategory={trench_category}")
        
        # Filter data from dfcombined based on modelVersionId
        combo_df = dfcombined[dfcombined['modelVersionId'] == model_version_id].copy()
        
        # If trenchCategory is not 'ALL', filter by it
        if trench_category != 'ALL':
            combo_df = combo_df[combo_df['trenchCategory'] == trench_category].copy()
        
        if len(combo_df) == 0:
            print(f"  Warning: No data found for this combination. Skipping...")
            continue
        
        print(f"  Data points: {len(combo_df)}")
        
        # Expand calcFeatures
        try:
            combo_df = expand_calc_features_robust(combo_df)
            if debug:
                print(f"  Features expanded successfully. Columns: {combo_df.shape[1]}")
        except Exception as e:
            print(f"  Error expanding features: {e}. Skipping...")
            continue
        
        # Extract feature list (all columns starting with modelVersionId_Calc_)
        feature_prefix = f"{model_version_id}_Calc_"
        feature_list = [col for col in combo_df.columns if col.startswith(feature_prefix)]
        
        # Also include 'score' if it exists
        if 'score' in combo_df.columns:
            feature_list.append('score')
        
        if len(feature_list) == 0:
            print(f"  Warning: No features found after expansion. Skipping...")
            continue
        
        print(f"  Features identified: {len(feature_list)}")
        
        # Define segment columns
        segment_columns = ['new_loan_type', 'osType', 'loan_product_type', 'trenchCategory']
        # Filter to only existing columns
        segment_columns = [col for col in segment_columns if col in combo_df.columns]
        
        # Calculate PSI (overall + segments + score)
        try:
            psi_result = calculate_month_on_month_psi(
                combo_df,
                feature_list,
                segment_columns=segment_columns
            )
            
            # Add model metadata
            psi_result['modelDisplayName'] = model_display_name
            psi_result['modelVersionId'] = model_version_id
            psi_result['trenchCategory'] = trench_category
            
            all_results.append(psi_result)
            print(f"  PSI calculated: {len(psi_result)} rows")
            
        except Exception as e:
            print(f"  Error calculating PSI: {e}")
            continue
        
    # Combine all results
    if all_results:
        final_result = pd.concat(all_results, ignore_index=True)
        
        # Reorder columns to match required output
        column_order = ['modelDisplayName', 'modelVersionId', 'trenchCategory',
                       'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                       'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                       'Expected_Percentage', 'Actual_Percentage', 'PSI', 'PSI_Interpretation']
        
        # Keep only columns that exist
        available_cols = [col for col in column_order if col in final_result.columns]
        final_result = final_result[available_cols]
        
        # Sort results
        final_result = final_result.sort_values(['modelVersionId', 'trenchCategory', 
                                                'Feature', 'Month', 'Segment_Column'])
        
        print(f"\n{'='*80}")
        print(f"Pipeline Complete!")
        print(f"Total rows in final output: {len(final_result)}")
        print(f"Unique combinations processed: {len(final_result[['modelVersionId', 'trenchCategory']].drop_duplicates())}")
        print(f"Unique features processed: {final_result['Feature'].nunique()}")
        print(f"{'='*80}")
        
        return final_result
    else:
        print("No results generated. Check input data and configurations.")
        return pd.DataFrame()


def expand_calc_features_robust(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expand the calcFeatures JSON column into separate columns.
    """
    df_expanded = df.copy()
    
    if 'calcFeatures' not in df_expanded.columns:
        print("Warning: 'calcFeatures' column not found in dataframe")
        return df_expanded
    
    # Get modelVersionId (assuming all rows have same modelVersionId in filtered dataframe)
    if len(df_expanded) == 0:
        return df_expanded
    
    model_version_id = df_expanded['modelVersionId'].iloc[0]
    prefix = f"{model_version_id}_Calc_"
    
    all_features_data = []
    
    for idx, row in df_expanded.iterrows():
        calc_features_str = row['calcFeatures']
        row_features = {}
        
        if pd.isna(calc_features_str):
            all_features_data.append(row_features)
            continue
            
        try:
            # Clean the string and parse JSON
            features_str = str(calc_features_str)
            # Handle common JSON issues
            features_str = features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            
            features_dict = json.loads(features_str)
            
            if isinstance(features_dict, dict):
                row_features = features_dict
            else:
                print(f"Warning: calcFeatures is not a dictionary at index {idx}")
                
        except json.JSONDecodeError as e:
            print(f"Warning: JSON decode error at index {idx}: {e}")
        except Exception as e:
            print(f"Warning: Error processing calcFeatures at index {idx}: {e}")
        
        all_features_data.append(row_features)
    
    # Create DataFrame from features
    features_df = pd.DataFrame(all_features_data)
    
    # Add prefix to column names
    features_df = features_df.add_prefix(prefix)
    
    # Combine with original dataframe
    df_expanded = df_expanded.reset_index(drop=True)
    features_df = features_df.reset_index(drop=True)
    
    result_df = pd.concat([df_expanded, features_df], axis=1)
    
    # Optionally drop the original calcFeatures column
    if 'calcFeatures' in result_df.columns:
        result_df = result_df.drop('calcFeatures', axis=1)
    
    return result_df


def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.
    """
    categorical_features = []
    numerical_features = []
    
    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature {feature} not found in dataframe")
            continue
        
        # Check if column exists and has data
        if df[feature].isnull().all():
            print(f"Warning: Feature {feature} is all null")
            continue
        
        # For numeric columns, check unique values
        if pd.api.types.is_numeric_dtype(df[feature]):
            unique_count = df[feature].nunique()
            
            # Check if it's likely categorical (few unique values and integers)
            if unique_count <= 10:
                sample_values = df[feature].dropna().head(100)
                # Check if values are essentially integers
                if all(abs(val - int(val)) < 0.0001 if not pd.isna(val) else True for val in sample_values):
                    categorical_features.append(feature)
                else:
                    numerical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            # Non-numeric columns are treated as categorical
            categorical_features.append(feature)
    
    print(f"Identified {len(numerical_features)} numerical and {len(categorical_features)} categorical features")
    
    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features and categorical features based on training period.
    """
    binning_info = {}
    
    # Process numerical features
    for feature in numerical_features:
        if feature not in train_period_df.columns:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        feature_data = train_period_df[feature].dropna()
        
        if len(feature_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        # Create bins based on percentiles
        try:
            # Try decile bins
            percentiles = np.percentile(feature_data, np.arange(0, 101, 10))
            percentiles = np.unique(percentiles)
            
            # Ensure we have at least 2 unique bins
            if len(percentiles) >= 2:
                bins = percentiles.copy()
                # Ensure first bin starts at -inf and last at inf
                bins[0] = -np.inf
                bins[-1] = np.inf
                
                # Create bin labels and ranges
                bin_ranges = {}
                for i in range(len(bins)-1):
                    bin_name = f"Bin_{i+1}"
                    lower = bins[i]
                    upper = bins[i+1]
                    
                    if np.isinf(lower) and np.isinf(upper):
                        range_str = "(-inf, inf)"
                    elif np.isinf(lower):
                        range_str = f"(-inf, {upper:.3f})"
                    elif np.isinf(upper):
                        range_str = f"[{lower:.3f}, inf)"
                    else:
                        range_str = f"[{lower:.3f}, {upper:.3f})"
                    
                    bin_ranges[bin_name] = {
                        'min': lower,
                        'max': upper,
                        'range_str': range_str
                    }
                
                binning_info[feature] = {
                    'type': 'numerical',
                    'bins': bins,
                    'bin_ranges': bin_ranges,
                    'bin_count': len(bins) - 1
                }
            else:
                # Fallback: simple min-max bins
                min_val = feature_data.min()
                max_val = feature_data.max()
                
                if min_val == max_val:
                    # Handle constant value
                    bins = np.array([-np.inf, min_val - 0.001, min_val, min_val + 0.001, np.inf])
                else:
                    bins = np.array([-np.inf, min_val, max_val, np.inf])
                
                bin_ranges = {
                    'Bin_1': {'min': -np.inf, 'max': min_val, 'range_str': f'(-inf, {min_val:.3f})'},
                    'Bin_2': {'min': min_val, 'max': max_val, 'range_str': f'[{min_val:.3f}, {max_val:.3f})'},
                    'Bin_3': {'min': max_val, 'max': np.inf, 'range_str': f'[{max_val:.3f}, inf)'}
                }
                
                binning_info[feature] = {
                    'type': 'numerical',
                    'bins': bins,
                    'bin_ranges': bin_ranges,
                    'bin_count': len(bins) - 1
                }
                
        except Exception as e:
            print(f"Error creating bins for {feature}: {e}")
            # Fallback to simple bins
            min_val = feature_data.min()
            max_val = feature_data.max()
            
            if min_val == max_val:
                bins = np.array([-np.inf, min_val - 0.001, min_val, min_val + 0.001, np.inf])
            else:
                bins = np.array([-np.inf, min_val, max_val, np.inf])
            
            binning_info[feature] = {
                'type': 'numerical',
                'bins': bins,
                'bin_ranges': {},
                'bin_count': len(bins) - 1
            }
    
    # Process categorical features
    for feature in categorical_features:
        if feature not in train_period_df.columns:
            binning_info[feature] = {'type': 'categorical', 'top_categories': [], 'bin_ranges': {}}
            continue
        
        value_counts = train_period_df[feature].value_counts()
        
        # Get top categories (up to 10)
        top_categories = value_counts.head(10).index.tolist()
        
        # Ensure 'Missing' is treated separately
        if 'Missing' in top_categories:
            top_categories.remove('Missing')
        
        binning_info[feature] = {
            'type': 'categorical',
            'top_categories': top_categories,
            'all_categories': value_counts.index.tolist(),
            'value_counts': value_counts.to_dict()
        }
    
    return binning_info


def apply_binning(df: pd.DataFrame, feature: str, binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.
    """
    if feature not in df.columns:
        return pd.Series(['Feature_Not_Found'] * len(df), index=df.index)
    
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None or len(binning_info['bins']) < 2:
            # Handle missing or invalid bins
            result = pd.Series(['No_Bins'] * len(df), index=df.index)
            result[df[feature].isna()] = 'Missing'
            return result
        
        bins = binning_info['bins']
        # Create labels
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
        try:
            # Apply binning
            binned = pd.cut(df[feature], bins=bins, labels=labels, 
                          include_lowest=True, duplicates='drop')
            binned = binned.astype(str)
            
            # Handle NaN values
            binned[df[feature].isna()] = 'Missing'
            
            # Handle any other issues
            binned[binned.isna()] = 'Other'
            
            return binned
        except Exception as e:
            print(f"Error binning {feature}: {e}")
            return pd.Series(['Binning_Error'] * len(df), index=df.index)
    
    else:  # Categorical
        top_categories = binning_info.get('top_categories', [])
        
        # Convert to string and handle NaN
        feature_data = df[feature].astype(str)
        feature_data[df[feature].isna()] = 'Missing'
        
        # Apply categorization
        def categorize_value(x):
            if x == 'Missing':
                return 'Missing'
            elif x in top_categories:
                return x
            elif str(x) in top_categories:
                return str(x)
            else:
                return 'Others'
        
        binned = feature_data.apply(categorize_value)
        return binned


def calculate_psi(expected_pct: pd.Series, actual_pct: pd.Series, 
                  epsilon: float = 1e-10) -> float:
    """
    Calculate Population Stability Index.
    """
    # Align both series on the same index
    all_bins = set(expected_pct.index) | set(actual_pct.index)
    
    # Create aligned series
    expected_aligned = pd.Series(0.0, index=list(all_bins))
    actual_aligned = pd.Series(0.0, index=list(all_bins))
    
    for idx in all_bins:
        if idx in expected_pct.index:
            expected_aligned[idx] = expected_pct[idx]
        if idx in actual_pct.index:
            actual_aligned[idx] = actual_pct[idx]
    
    # Apply epsilon to avoid zeros
    expected_aligned = expected_aligned.apply(lambda x: max(x, epsilon))
    actual_aligned = actual_aligned.apply(lambda x: max(x, epsilon))
    
    # Normalize to sum to 1
    expected_aligned = expected_aligned / expected_aligned.sum()
    actual_aligned = actual_aligned / actual_aligned.sum()
    
    # Calculate PSI
    psi_value = np.sum((actual_aligned - expected_aligned) * 
                      np.log(actual_aligned / expected_aligned))
    
    return psi_value


def interpret_psi(psi_value: float) -> str:
    """
    Interpret PSI value based on industry standards.
    """
    if psi_value < 0.1:
        return "Very Stable"
    elif psi_value < 0.2:
        return "Stable"
    elif psi_value < 0.5:
        return "Moderate Shift"
    else:
        return "Significant Shift"


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str] = [],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period vs each month,
    with overall and segment-level breakdowns.
    """
    df = df.copy()
    
    # Separate train and test data
    train_mask = df[data_selection_col] == 'Train'
    test_mask = df[data_selection_col] != 'Train'
    
    train_df = df[train_mask].copy()
    test_df = df[test_mask].copy()
    
    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")
    
    if len(test_df) == 0:
        raise ValueError("No test data found. Check Data_selection column.")
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Create bins based on training data
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'], 
        feature_types['categorical'], 
        train_df
    )
    
    # Get test months
    test_months = sorted(test_df[month_col].unique())
    
    # Store results
    results = []
    
    # Create temporary binned columns
    temp_columns = []
    for feature in feature_list:
        if feature not in binning_info:
            continue
            
        binned_col = f'{feature}_binned'
        df[binned_col] = apply_binning(df, feature, binning_info[feature])
        temp_columns.append(binned_col)
    
    # 1. Calculate overall PSI (no segments)
    print("  Calculating overall PSI...")
    for feature in feature_list:
        if feature not in binning_info:
            continue
            
        binned_col = f'{feature}_binned'
        
        # Get baseline distribution from ALL training data
        train_baseline = df[train_mask][binned_col].value_counts(normalize=True)
        
        # Skip if baseline is empty
        if len(train_baseline) == 0:
            continue
        
        # Get total training count (across all training months)
        total_train_count = train_df[account_id_col].nunique()
        
        for month in test_months:
            month_mask = df[month_col] == month
            test_month_mask = month_mask & test_mask
            
            # Get current month distribution
            actual_dist = df[test_month_mask][binned_col].value_counts(normalize=True)
            
            # Skip if no data for this month
            if len(actual_dist) == 0:
                continue
            
            # Calculate PSI
            psi_value = calculate_psi(train_baseline, actual_dist)
            
            # Get counts
            base_count = total_train_count  # Total training data count
            actual_count = df[test_month_mask][account_id_col].nunique()
            
            # Get distribution statistics
            expected_avg_pct = (train_baseline * 100).mean() if len(train_baseline) > 0 else 0
            actual_avg_pct = (actual_dist * 100).mean() if len(actual_dist) > 0 else 0
            
            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': month,
                'Base_Month': 'All_Training_Months',
                'Current_Month': month,
                'Base_Count': base_count,
                'Actual_Count': actual_count,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value,
                'PSI_Interpretation': interpret_psi(psi_value)
            })
    
    # 2. Calculate segment-level PSI
    print("  Calculating segment-level PSI...")
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
            
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_mask = df[segment_col] == segment_val
            
            # Skip if segment has no data
            if not segment_mask.any():
                continue
            
            for feature in feature_list:
                if feature not in binning_info:
                    continue
                    
                binned_col = f'{feature}_binned'
                
                # Get training baseline for this segment (ALL training data for this segment)
                train_segment_mask = train_mask & segment_mask
                
                if not train_segment_mask.any():
                    continue
                    
                train_baseline = df[train_segment_mask][binned_col].value_counts(normalize=True)
                
                if len(train_baseline) == 0:
                    continue
                
                # Get training count for this segment
                segment_train_count = df[train_segment_mask][account_id_col].nunique()
                
                for month in test_months:
                    month_segment_mask = (df[month_col] == month) & segment_mask & test_mask
                    
                    if not month_segment_mask.any():
                        continue
                    
                    actual_dist = df[month_segment_mask][binned_col].value_counts(normalize=True)
                    
                    if len(actual_dist) == 0:
                        continue
                    
                    # Calculate PSI
                    psi_value = calculate_psi(train_baseline, actual_dist)
                    
                    # Get counts
                    base_count = segment_train_count
                    actual_count = df[month_segment_mask][account_id_col].nunique()
                    
                    # Get distribution statistics
                    expected_avg_pct = (train_baseline * 100).mean() if len(train_baseline) > 0 else 0
                    actual_avg_pct = (actual_dist * 100).mean() if len(actual_dist) > 0 else 0
                    
                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': str(segment_val),
                        'Month': month,
                        'Base_Month': 'All_Training_Months',
                        'Current_Month': month,
                        'Base_Count': base_count,
                        'Actual_Count': actual_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value,
                        'PSI_Interpretation': interpret_psi(psi_value)
                    })
    
    # Clean up temporary columns
    for col in temp_columns:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
    
    return pd.DataFrame(results)


# ============================================================================
# CORRECTED VALIDATION FUNCTION
# ============================================================================

def validate_psi_counts(psi_results: pd.DataFrame, dfcombined: pd.DataFrame) -> pd.DataFrame:
    """
    Validate the counts in PSI results against the original data.
    """
    validation_results = []
    
    for idx, row in psi_results.iterrows():
        model_display_name = row['modelDisplayName']
        model_version_id = row['modelVersionId']
        config_trench = row['trenchCategory']  # This is from config: can be 'ALL' or specific
        month = row['Month']
        segment_col = row['Segment_Column']
        segment_val = row['Segment_Value']
        
        try:
            # Build base filters for model
            base_filters = [
                f"modelDisplayName == '{model_display_name}'",
                f"modelVersionId == '{model_version_id}'"
            ]
            
            # Handle trenchCategory from config
            if config_trench != 'ALL':
                # If config has specific trench, filter by it
                base_filters.append(f"trenchCategory == '{config_trench}'")
            # If config_trench is 'ALL', we don't filter by trenchCategory
            
            # For segment-specific validation
            if segment_col != 'Overall':
                # Add segment filter
                base_filters.append(f"{segment_col} == '{segment_val}'")
            
            # TRAINING DATA COUNT
            # Training data is all data with Data_selection = 'Train'
            train_filters = base_filters + ["Data_selection == 'Train'"]
            
            # Build query for training data
            train_query = ' & '.join(train_filters)
            train_data = dfcombined.query(train_query)
            actual_train_count = train_data['digitalLoanAccountId'].nunique()
            
            # TEST DATA COUNT
            # Test data is for specific month and Data_selection != 'Train'
            test_filters = base_filters + [
                f"Application_month == '{month}'",
                "Data_selection != 'Train'"
            ]
            
            # Build query for test data
            test_query = ' & '.join(test_filters)
            test_data = dfcombined.query(test_query)
            actual_test_count = test_data['digitalLoanAccountId'].nunique()
            
            # Compare with PSI results
            validation = {
                'Row_Index': idx,
                'Model': model_display_name,
                'ModelVersion': model_version_id,
                'Config_Trench': config_trench,
                'Month': month,
                'Segment_Column': segment_col,
                'Segment_Value': segment_val,
                'PSI_Base_Count': row['Base_Count'],
                'Actual_Train_Count': actual_train_count,
                'PSI_Actual_Count': row['Actual_Count'],
                'Actual_Test_Count': actual_test_count,
                'Train_Match': row['Base_Count'] == actual_train_count,
                'Test_Match': row['Actual_Count'] == actual_test_count,
                'Train_Difference': row['Base_Count'] - actual_train_count,
                'Test_Difference': row['Actual_Count'] - actual_test_count
            }
            
            validation_results.append(validation)
            
        except Exception as e:
            print(f"Error validating row {idx}: {e}")
            validation = {
                'Row_Index': idx,
                'Model': model_display_name,
                'ModelVersion': model_version_id,
                'Config_Trench': config_trench,
                'Month': month,
                'Segment_Column': segment_col,
                'Segment_Value': segment_val,
                'PSI_Base_Count': row['Base_Count'],
                'Actual_Train_Count': 'ERROR',
                'PSI_Actual_Count': row['Actual_Count'],
                'Actual_Test_Count': 'ERROR',
                'Train_Match': False,
                'Test_Match': False,
                'Train_Difference': 'ERROR',
                'Test_Difference': 'ERROR',
                'Error': str(e)
            }
            validation_results.append(validation)
    
    return pd.DataFrame(validation_results)


# ============================================================================
# QUICK FIX FUNCTION FOR YOUR SPECIFIC ISSUE
# ============================================================================

def debug_counts(dfcombined: pd.DataFrame, 
                 model_display_name: str,
                 model_version_id: str,
                 trench_category: str,
                 month: str) -> pd.DataFrame:
    """
    Debug function to check counts for a specific combination.
    """
    print(f"\nDebugging counts for:")
    print(f"  Model: {model_display_name}")
    print(f"  Version: {model_version_id}")
    print(f"  Trench (config): {trench_category}")
    print(f"  Month: {month}")
    
    # Build filters
    filters = [
        f"modelDisplayName == '{model_display_name}'",
        f"modelVersionId == '{model_version_id}'"
    ]
    
    # Handle trench category
    if trench_category != 'ALL':
        filters.append(f"trenchCategory == '{trench_category}'")
    
    # Training data count
    train_filters = filters + ["Data_selection == 'Train'"]
    train_query = ' & '.join(train_filters)
    train_data = dfcombined.query(train_query)
    train_count = train_data['digitalLoanAccountId'].nunique()
    
    print(f"\nTraining data query: {train_query}")
    print(f"Training count: {train_count}")
    
    # Test data count for specific month
    test_filters = filters + [
        f"Application_month == '{month}'",
        "Data_selection != 'Train'"
    ]
    test_query = ' & '.join(test_filters)
    test_data = dfcombined.query(test_query)
    test_count = test_data['digitalLoanAccountId'].nunique()
    
    print(f"\nTest data query: {test_query}")
    print(f"Test count for {month}: {test_count}")
    
    # Show sample data
    print(f"\nSample training data (first 5 rows):")
    print(train_data[['digitalLoanAccountId', 'Application_month', 'Data_selection', 'trenchCategory']].head())
    
    print(f"\nSample test data for {month} (first 5 rows):")
    print(test_data[['digitalLoanAccountId', 'Application_month', 'Data_selection', 'trenchCategory']].head())
    
    return pd.DataFrame({
        'query_type': ['training', 'test'],
        'query': [train_query, test_query],
        'count': [train_count, test_count]
    })


# # ============================================================================
# # USAGE EXAMPLE WITH DEBUG
# # ============================================================================

# def main():
#     """
#     Example usage with debug and validation.
#     """
#     # Load your data
#     # dfcombined = pd.read_csv('your_data.csv')
#     # configdf = pd.read_csv('your_config.csv')
    
#     # Example data structure (you should replace with your actual data)
#     dfcombined = pd.DataFrame()  # Your actual dataframe
#     configdf = pd.DataFrame()    # Your actual config
    
#     # First, debug a specific case
#     print("="*80)
#     print("DEBUGGING SPECIFIC CASE")
#     print("="*80)
    
#     debug_result = debug_counts(
#         dfcombined=dfcombined,
#         model_display_name='cic_model_sil',
#         model_version_id='v2',
#         trench_category='ALL',
#         month='2025-11'
#     )
    
#     print(f"\nDebug result:\n{debug_result}")
    
#     # Calculate PSI
#     print("\n" + "="*80)
#     print("CALCULATING PSI")
#     print("="*80)
    
#     psi_results = calculate_psi_for_model(
#         dfcombined=dfcombined,
#         configdf=configdf,
#         model_display_name='cic_model_sil',
#         debug=True
#     )
    
#     if not psi_results.empty:
#         # Save results
#         psi_results.to_csv('psi_results.csv', index=False)
#         print(f"\nResults saved to psi_results.csv")
        
#         # Validate counts
#         print("\n" + "="*80)
#         print("VALIDATING COUNTS")
#         print("="*80)
        
#         validation = validate_psi_counts(psi_results, dfcombined)
        
#         # Check for mismatches
#         mismatches = validation[(validation['Train_Match'] == False) | (validation['Test_Match'] == False)]
        
#         if len(mismatches) > 0:
#             print(f"\nFound {len(mismatches)} count mismatches:")
#             print(mismatches.head(20))
            
#             # Save mismatches to CSV for detailed analysis
#             mismatches.to_csv('psi_validation_mismatches.csv', index=False)
#             print("\nDetailed mismatches saved to psi_validation_mismatches.csv")
            
#             # Show summary of mismatches
#             print("\nSummary of mismatches:")
#             mismatch_summary = mismatches.groupby(['Model', 'ModelVersion', 'Config_Trench']).agg({
#                 'Train_Match': lambda x: (x == False).sum(),
#                 'Test_Match': lambda x: (x == False).sum(),
#                 'Train_Difference': 'mean',
#                 'Test_Difference': 'mean'
#             }).reset_index()
            
#             print(mismatch_summary)
#         else:
#             print("✓ All counts match!")
        
#         # Summary statistics
#         print("\n" + "="*80)
#         print("PSI SUMMARY")
#         print("="*80)
        
#         print(f"Total calculations: {len(psi_results):,}")
#         print(f"Average PSI: {psi_results['PSI'].mean():.4f}")
#         print(f"Maximum PSI: {psi_results['PSI'].max():.4f}")
        
#         # Distribution of PSI interpretations
#         print(f"\nPSI Interpretation Distribution:")
#         print(psi_results['PSI_Interpretation'].value_counts())
        
#         # Features with highest PSI
#         high_psi = psi_results[psi_results['PSI'] > 0.2]
#         if len(high_psi) > 0:
#             print(f"\nFeatures with PSI > 0.2 (moderate or significant shift): {len(high_psi):,}")
#             top_features = high_psi.groupby('Feature')['PSI'].max().sort_values(ascending=False).head(10)
#             print("\nTop 10 features with highest PSI:")
#             for feature, psi_val in top_features.items():
#                 print(f"  {feature}: {psi_val:.4f}")
    
#     return psi_results


# if __name__ == "__main__":
#     # Run the analysis
#     results = main()

### PSI pipeline Version 2

In [6]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional, Union
import warnings
import json
warnings.filterwarnings('ignore')


def calculate_psi_for_model(dfcombined: pd.DataFrame,
                            configdf: pd.DataFrame,
                            model_display_name: str,
                            debug: bool = False) -> pd.DataFrame:
    """
    Calculate PSI for a specific model based on configdf combinations.
    """
    
    # Filter configdf to only include combinations for this specific modelDisplayName
    model_config = configdf[configdf['modelDisplayName'] == model_display_name].copy()
    
    print(f"\n{'='*80}")
    print(f"Starting PSI Pipeline for Model: {model_display_name}")
    print(f"Total combinations to process: {len(model_config)}")
    print(f"{'='*80}\n")
    
    if len(model_config) == 0:
        print(f"ERROR: No configurations found for modelDisplayName={model_display_name}")
        return pd.DataFrame()
    
    # Process each unique combination
    all_results = []
    
    for idx, config_row in model_config.iterrows():
        model_version_id = config_row['modelVersionId']
        trench_category = config_row['trenchCategory']
        
        print(f"Processing combination {idx + 1}/{len(model_config)}: "
              f"modelVersionId={model_version_id}, trenchCategory={trench_category}")
        
        # Filter data from dfcombined based on modelVersionId
        combo_df = dfcombined[dfcombined['modelVersionId'] == model_version_id].copy()
        
        # If trenchCategory is not 'ALL', filter by it
        if trench_category != 'ALL':
            combo_df = combo_df[combo_df['trenchCategory'] == trench_category].copy()
        
        if len(combo_df) == 0:
            print(f"  Warning: No data found for this combination. Skipping...")
            continue
        
        print(f"  Data points: {len(combo_df)}")
        
        # Expand calcFeatures
        try:
            combo_df = expand_calc_features_robust(combo_df)
            if debug:
                print(f"  Features expanded successfully. Columns: {combo_df.shape[1]}")
        except Exception as e:
            print(f"  Error expanding features: {e}. Skipping...")
            continue
        
        # Extract feature list (all columns starting with modelVersionId_Calc_)
        feature_prefix = f"{model_version_id}_Calc_"
        feature_list = [col for col in combo_df.columns if col.startswith(feature_prefix)]
        
        # Also include 'score' if it exists
        if 'score' in combo_df.columns:
            feature_list.append('score')
        
        if len(feature_list) == 0:
            print(f"  Warning: No features found after expansion. Skipping...")
            continue
        
        print(f"  Features identified: {len(feature_list)}")
        
        # Define segment columns
        segment_columns = ['new_loan_type', 'osType', 'loan_product_type']
        # Filter to only existing columns
        segment_columns = [col for col in segment_columns if col in combo_df.columns]
        
        # Calculate PSI (overall + segments + score)
        try:
            psi_result = calculate_month_on_month_psi(
                combo_df,
                feature_list,
                segment_columns=segment_columns
            )
            
            # Add model metadata
            psi_result['modelDisplayName'] = model_display_name
            psi_result['modelVersionId'] = model_version_id
            psi_result['trenchCategory'] = trench_category
            
            all_results.append(psi_result)
            print(f"  PSI calculated: {len(psi_result)} rows")
            
        except Exception as e:
            print(f"  Error calculating PSI: {e}")
            continue
        
    # Combine all results
    if all_results:
        final_result = pd.concat(all_results, ignore_index=True)
        
        # Reorder columns to match required output
        column_order = ['modelDisplayName', 'modelVersionId', 'trenchCategory',
                       'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                       'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                       'Expected_Percentage', 'Actual_Percentage', 'PSI', 'PSI_Interpretation']
        
        # Keep only columns that exist
        available_cols = [col for col in column_order if col in final_result.columns]
        final_result = final_result[available_cols]
        
        # Sort results
        final_result = final_result.sort_values(['modelVersionId', 'trenchCategory', 
                                                'Feature', 'Month', 'Segment_Column'])
        
        print(f"\n{'='*80}")
        print(f"Pipeline Complete!")
        print(f"Total rows in final output: {len(final_result)}")
        print(f"Unique combinations processed: {len(final_result[['modelVersionId', 'trenchCategory']].drop_duplicates())}")
        print(f"Unique features processed: {final_result['Feature'].nunique()}")
        print(f"{'='*80}")
        
        return final_result
    else:
        print("No results generated. Check input data and configurations.")
        return pd.DataFrame()


def expand_calc_features_robust(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expand the calcFeatures JSON column into separate columns.
    """
    df_expanded = df.copy()
    
    if 'calcFeatures' not in df_expanded.columns:
        print("Warning: 'calcFeatures' column not found in dataframe")
        return df_expanded
    
    # Get modelVersionId (assuming all rows have same modelVersionId in filtered dataframe)
    if len(df_expanded) == 0:
        return df_expanded
    
    model_version_id = df_expanded['modelVersionId'].iloc[0]
    prefix = f"{model_version_id}_Calc_"
    
    all_features_data = []
    
    for idx, row in df_expanded.iterrows():
        calc_features_str = row['calcFeatures']
        row_features = {}
        
        if pd.isna(calc_features_str):
            all_features_data.append(row_features)
            continue
            
        try:
            # Clean the string and parse JSON
            features_str = str(calc_features_str)
            # Handle common JSON issues
            features_str = features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            
            features_dict = json.loads(features_str)
            
            if isinstance(features_dict, dict):
                row_features = features_dict
            else:
                print(f"Warning: calcFeatures is not a dictionary at index {idx}")
                
        except json.JSONDecodeError as e:
            print(f"Warning: JSON decode error at index {idx}: {e}")
        except Exception as e:
            print(f"Warning: Error processing calcFeatures at index {idx}: {e}")
        
        all_features_data.append(row_features)
    
    # Create DataFrame from features
    features_df = pd.DataFrame(all_features_data)
    
    # Add prefix to column names
    features_df = features_df.add_prefix(prefix)
    
    # Combine with original dataframe
    df_expanded = df_expanded.reset_index(drop=True)
    features_df = features_df.reset_index(drop=True)
    
    result_df = pd.concat([df_expanded, features_df], axis=1)
    
    # Optionally drop the original calcFeatures column
    if 'calcFeatures' in result_df.columns:
        result_df = result_df.drop('calcFeatures', axis=1)
    
    return result_df


def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.
    """
    categorical_features = []
    numerical_features = []
    
    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature {feature} not found in dataframe")
            continue
        
        # Check if column exists and has data
        if df[feature].isnull().all():
            print(f"Warning: Feature {feature} is all null")
            continue
        
        # For numeric columns, check unique values
        if pd.api.types.is_numeric_dtype(df[feature]):
            unique_count = df[feature].nunique()
            
            # Check if it's likely categorical (few unique values and integers)
            if unique_count <= 10:
                sample_values = df[feature].dropna().head(100)
                # Check if values are essentially integers
                if all(abs(val - int(val)) < 0.0001 if not pd.isna(val) else True for val in sample_values):
                    categorical_features.append(feature)
                else:
                    numerical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            # Non-numeric columns are treated as categorical
            categorical_features.append(feature)
    
    print(f"Identified {len(numerical_features)} numerical and {len(categorical_features)} categorical features")
    
    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features and categorical features based on training period.
    
    For numerical features: Create decile bins (10 equal-frequency bins) based on training data
    For categorical features: Use exact categories from training data
    """
    binning_info = {}
    
    # Process numerical features - DECILE BINNING
    for feature in numerical_features:
        if feature not in train_period_df.columns:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        feature_data = train_period_df[feature].dropna()
        
        if len(feature_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        # CREATE DECILE BINS (10 equal-frequency bins)
        try:
            # Calculate decile thresholds (0th to 100th percentile in steps of 10)
            deciles = np.percentile(feature_data, np.arange(0, 101, 10))
            
            # Ensure unique values for bin edges
            unique_deciles = np.unique(deciles)
            
            # Handle edge case where all deciles might be the same
            if len(unique_deciles) == 1:
                # Single value - create symmetric bins around the value
                unique_deciles = np.array([
                    -np.inf, 
                    unique_deciles[0] - 0.001, 
                    unique_deciles[0], 
                    unique_deciles[0] + 0.001, 
                    np.inf
                ])
            
            # Create bins with -inf and inf for the boundaries
            bins = unique_deciles.copy()
            bins[0] = -np.inf
            bins[-1] = np.inf
            
            # Create bin labels and ranges
            bin_ranges = {}
            for i in range(len(bins)-1):
                bin_name = f"Bin_{i+1}"
                lower = bins[i]
                upper = bins[i+1]
                
                # Create descriptive range string
                if i == 0:  # First bin
                    range_str = f"(-inf, {upper:.6f})"
                elif i == len(bins)-2:  # Last bin
                    range_str = f"[{lower:.6f}, inf)"
                else:
                    # For middle bins, check if it's essentially a single value
                    if abs(upper - lower) < 1e-10:
                        range_str = f"[{lower:.6f}]"
                    else:
                        range_str = f"[{lower:.6f}, {upper:.6f})"
                
                bin_ranges[bin_name] = {
                    'min': lower,
                    'max': upper,
                    'range_str': range_str
                }
            
            # Store binning information
            binning_info[feature] = {
                'type': 'numerical',
                'bins': bins,
                'bin_ranges': bin_ranges,
                'bin_count': len(bins) - 1,
                'decile_values': deciles.tolist()  # Store decile values for debugging
            }
            
            if len(feature_data) > 0:
                print(f"    Created {len(bins)-1} decile bins for {feature} "
                      f"(min={feature_data.min():.4f}, max={feature_data.max():.4f})")
                
        except Exception as e:
            print(f"Error creating decile bins for {feature}: {e}")
            # Fallback to simple min-max bins
            try:
                min_val = feature_data.min()
                max_val = feature_data.max()
                
                if min_val == max_val:
                    bins = np.array([-np.inf, min_val - 0.001, min_val, min_val + 0.001, np.inf])
                    bin_count = 4
                else:
                    # Create 10 equal-width bins as fallback
                    bins = np.linspace(min_val, max_val, 11)
                    bins[0] = -np.inf
                    bins[-1] = np.inf
                    bin_count = 10
                
                binning_info[feature] = {
                    'type': 'numerical',
                    'bins': bins,
                    'bin_ranges': {},
                    'bin_count': bin_count
                }
                print(f"    Used fallback bins for {feature}")
            except:
                binning_info[feature] = {
                    'type': 'numerical',
                    'bins': None,
                    'bin_ranges': {},
                    'bin_count': 0
                }
    
    # Process categorical features - EXACT CATEGORY MATCHING
    for feature in categorical_features:
        if feature not in train_period_df.columns:
            binning_info[feature] = {'type': 'categorical', 'categories': [], 'value_counts': {}}
            continue
        
        # Get value counts from training data
        train_values = train_period_df[feature].dropna()
        value_counts = train_values.value_counts()
        
        # Store ALL unique categories from training data
        all_categories = value_counts.index.tolist()
        
        # Also store value counts for reference
        value_counts_dict = value_counts.to_dict()
        
        binning_info[feature] = {
            'type': 'categorical',
            'categories': all_categories,  # Store all categories from training
            'value_counts': value_counts_dict,
            'category_count': len(all_categories)
        }
        
        print(f"    Found {len(all_categories)} unique categories for {feature}")
    
    return binning_info


def apply_binning(df: pd.DataFrame, feature: str, binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.
    
    For numerical: Use decile bins created from training data
    For categorical: Use exact categories from training data, mark new values in test as 'Others'
    """
    if feature not in df.columns:
        return pd.Series(['Feature_Not_Found'] * len(df), index=df.index)
    
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None or len(binning_info['bins']) < 2:
            # Handle missing or invalid bins
            result = pd.Series(['No_Bins'] * len(df), index=df.index)
            result[df[feature].isna()] = 'Missing'
            return result
        
        bins = binning_info['bins']
        # Create labels for decile bins
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
        try:
            # Apply binning using training bins
            binned = pd.cut(df[feature], bins=bins, labels=labels, 
                          include_lowest=True, right=False, duplicates='drop')
            binned = binned.astype(str)
            
            # Handle NaN values
            binned[df[feature].isna()] = 'Missing'
            
            # Handle any values that couldn't be binned
            binned[binned.isna()] = 'Out_of_Range'
            
            return binned
        except Exception as e:
            print(f"Error binning {feature}: {e}")
            return pd.Series(['Binning_Error'] * len(df), index=df.index)
    
    else:  # Categorical
        train_categories = set(binning_info.get('categories', []))
        
        # Convert to string and handle NaN
        feature_data = df[feature].astype(str)
        feature_data[df[feature].isna()] = 'Missing'
        
        # Apply exact category matching
        def categorize_value(x):
            if x == 'Missing':
                return 'Missing'
            elif x in train_categories or str(x) in train_categories:
                # Return exact value if it exists in training data
                return x
            else:
                # Mark as 'Others' if not in training categories
                return 'Others'
        
        binned = feature_data.apply(categorize_value)
        return binned


def calculate_psi(expected_pct: pd.Series, actual_pct: pd.Series, 
                  epsilon: float = 1e-10) -> float:
    """
    Calculate Population Stability Index.
    """
    # Align both series on the same index
    all_bins = set(expected_pct.index) | set(actual_pct.index)
    
    # Create aligned series
    expected_aligned = pd.Series(0.0, index=list(all_bins))
    actual_aligned = pd.Series(0.0, index=list(all_bins))
    
    for idx in all_bins:
        if idx in expected_pct.index:
            expected_aligned[idx] = expected_pct[idx]
        if idx in actual_pct.index:
            actual_aligned[idx] = actual_pct[idx]
    
    # Apply epsilon to avoid zeros
    expected_aligned = expected_aligned.apply(lambda x: max(x, epsilon))
    actual_aligned = actual_aligned.apply(lambda x: max(x, epsilon))
    
    # Normalize to sum to 1
    expected_aligned = expected_aligned / expected_aligned.sum()
    actual_aligned = actual_aligned / actual_aligned.sum()
    
    # Calculate PSI
    psi_value = np.sum((actual_aligned - expected_aligned) * 
                      np.log(actual_aligned / expected_aligned))
    
    return psi_value


def interpret_psi(psi_value: float) -> str:
    """
    Interpret PSI value based on industry standards.
    """
    if psi_value < 0.1:
        return "Very Stable"
    elif psi_value < 0.2:
        return "Stable"
    elif psi_value < 0.5:
        return "Moderate Shift"
    else:
        return "Significant Shift"


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str] = [],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period vs each month,
    with overall and segment-level breakdowns.
    
    Uses decile binning for numerical features and exact category matching
    for categorical features from training to test.
    """
    df = df.copy()
    
    # Separate train and test data
    train_mask = df[data_selection_col] == 'Train'
    test_mask = df[data_selection_col] != 'Train'
    
    train_df = df[train_mask].copy()
    test_df = df[test_mask].copy()
    
    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")
    
    if len(test_df) == 0:
        raise ValueError("No test data found. Check Data_selection column.")
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Create bins based on training data (DECILE BINNING for numerical, EXACT CATEGORIES for categorical)
    print("  Creating decile bins from training data...")
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'], 
        feature_types['categorical'], 
        train_df
    )
    
    # Get test months
    test_months = sorted(test_df[month_col].unique())
    
    # Store results
    results = []
    
    # Create temporary binned columns for all data (train + test)
    # This ensures consistent binning using training bins
    temp_columns = []
    for feature in feature_list:
        if feature not in binning_info:
            continue
            
        binned_col = f'{feature}_binned'
        df[binned_col] = apply_binning(df, feature, binning_info[feature])
        temp_columns.append(binned_col)
    
    # 1. Calculate overall PSI (no segments)
    print("  Calculating overall PSI...")
    for feature in feature_list:
        if feature not in binning_info:
            continue
            
        binned_col = f'{feature}_binned'
        
        # Get baseline distribution from ALL training data
        train_baseline = df[train_mask][binned_col].value_counts(normalize=True)
        
        # Skip if baseline is empty
        if len(train_baseline) == 0:
            continue
        
        # Get total training count (across all training months)
        total_train_count = train_df[account_id_col].nunique()
        
        # Calculate distribution percentages for training data
        train_counts = df[train_mask][binned_col].value_counts()
        train_total = train_counts.sum()
        
        for month in test_months:
            month_mask = df[month_col] == month
            test_month_mask = month_mask & test_mask
            
            # Skip if no test data for this month
            if not test_month_mask.any():
                continue
            
            # Get current month distribution
            actual_counts = df[test_month_mask][binned_col].value_counts()
            actual_total = actual_counts.sum()
            
            # Skip if no data for this month
            if actual_total == 0:
                continue
            
            # Calculate normalized percentages
            actual_dist = actual_counts / actual_total
            
            # Calculate PSI using the distributions
            psi_value = calculate_psi(train_baseline, actual_dist)
            
            # Get distribution statistics
            expected_avg_pct = (train_baseline * 100).mean() if len(train_baseline) > 0 else 0
            actual_avg_pct = (actual_dist * 100).mean() if len(actual_dist) > 0 else 0
            
            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': month,
                'Base_Month': 'All_Training_Months',
                'Current_Month': month,
                'Base_Count': total_train_count,
                'Actual_Count': df[test_month_mask][account_id_col].nunique(),
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value,
                'PSI_Interpretation': interpret_psi(psi_value),
                'Bin_Count': binning_info[feature].get('bin_count', 
                          binning_info[feature].get('category_count', 0))
            })
    
    # 2. Calculate segment-level PSI
    print("  Calculating segment-level PSI...")
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
            
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_mask = df[segment_col] == segment_val
            
            # Skip if segment has no data
            if not segment_mask.any():
                continue
            
            for feature in feature_list:
                if feature not in binning_info:
                    continue
                    
                binned_col = f'{feature}_binned'
                
                # Get training baseline for this segment (ALL training data for this segment)
                train_segment_mask = train_mask & segment_mask
                
                if not train_segment_mask.any():
                    continue
                    
                train_baseline = df[train_segment_mask][binned_col].value_counts(normalize=True)
                
                if len(train_baseline) == 0:
                    continue
                
                # Get training count for this segment
                segment_train_count = df[train_segment_mask][account_id_col].nunique()
                
                for month in test_months:
                    month_segment_mask = (df[month_col] == month) & segment_mask & test_mask
                    
                    if not month_segment_mask.any():
                        continue
                    
                    actual_counts = df[month_segment_mask][binned_col].value_counts()
                    actual_total = actual_counts.sum()
                    
                    if actual_total == 0:
                        continue
                    
                    actual_dist = actual_counts / actual_total
                    
                    # Calculate PSI
                    psi_value = calculate_psi(train_baseline, actual_dist)
                    
                    # Get counts
                    base_count = segment_train_count
                    actual_count = df[month_segment_mask][account_id_col].nunique()
                    
                    # Get distribution statistics
                    expected_avg_pct = (train_baseline * 100).mean() if len(train_baseline) > 0 else 0
                    actual_avg_pct = (actual_dist * 100).mean() if len(actual_dist) > 0 else 0
                    
                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': str(segment_val),
                        'Month': month,
                        'Base_Month': 'All_Training_Months',
                        'Current_Month': month,
                        'Base_Count': base_count,
                        'Actual_Count': actual_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value,
                        'PSI_Interpretation': interpret_psi(psi_value),
                        'Bin_Count': binning_info[feature].get('bin_count', 
                                  binning_info[feature].get('category_count', 0))
                    })
    
    # Clean up temporary columns
    for col in temp_columns:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
    
    return pd.DataFrame(results)


# ============================================================================
# ADDITIONAL HELPER FUNCTIONS FOR DECILE BINNING ANALYSIS
# ============================================================================

def analyze_decile_bins(df: pd.DataFrame, feature: str, train_mask: pd.Series) -> Dict:
    """
    Analyze decile bins for a numerical feature.
    Returns detailed information about the decile bins.
    """
    train_data = df[train_mask][feature].dropna()
    
    if len(train_data) == 0:
        return {}
    
    # Calculate deciles
    deciles = np.percentile(train_data, np.arange(0, 101, 10))
    
    # Create bins
    bins = np.unique(deciles)
    bins[0] = -np.inf
    bins[-1] = np.inf
    
    # Calculate bin statistics
    bin_stats = []
    for i in range(len(bins)-1):
        lower = bins[i]
        upper = bins[i+1]
        
        if i == 0:
            mask = train_data < upper
        elif i == len(bins)-2:
            mask = train_data >= lower
        else:
            mask = (train_data >= lower) & (train_data < upper)
        
        bin_data = train_data[mask]
        
        bin_stats.append({
            'bin': i+1,
            'lower_bound': lower,
            'upper_bound': upper,
            'count': len(bin_data),
            'percentage': len(bin_data) / len(train_data) * 100,
            'min_in_bin': bin_data.min() if len(bin_data) > 0 else None,
            'max_in_bin': bin_data.max() if len(bin_data) > 0 else None,
            'mean_in_bin': bin_data.mean() if len(bin_data) > 0 else None
        })
    
    return {
        'feature': feature,
        'decile_values': deciles.tolist(),
        'bins': bins.tolist(),
        'bin_stats': bin_stats,
        'total_count': len(train_data),
        'min_value': train_data.min(),
        'max_value': train_data.max(),
        'mean_value': train_data.mean()
    }


def validate_categorical_matching(df: pd.DataFrame, feature: str, 
                                  train_mask: pd.Series, test_mask: pd.Series) -> Dict:
    """
    Validate categorical feature matching between train and test.
    """
    train_cats = set(df[train_mask][feature].dropna().unique())
    test_cats = set(df[test_mask][feature].dropna().unique())
    
    categories_only_in_test = test_cats - train_cats
    categories_only_in_train = train_cats - test_cats
    common_categories = train_cats.intersection(test_cats)
    
    return {
        'feature': feature,
        'train_categories_count': len(train_cats),
        'test_categories_count': len(test_cats),
        'common_categories_count': len(common_categories),
        'categories_only_in_test': list(categories_only_in_test),
        'categories_only_in_train': list(categories_only_in_train),
        'categories_only_in_test_count': len(categories_only_in_test),
        'categories_only_in_train_count': len(categories_only_in_train)
    }


def debug_binning_strategy(dfcombined: pd.DataFrame, 
                          model_display_name: str,
                          model_version_id: str,
                          feature_name: str) -> pd.DataFrame:
    """
    Debug function to analyze binning strategy for a specific feature.
    """
    print(f"\nDebugging binning strategy for:")
    print(f"  Model: {model_display_name}")
    print(f"  Version: {model_version_id}")
    print(f"  Feature: {feature_name}")
    
    # Filter data
    df = dfcombined[dfcombined['modelVersionId'] == model_version_id].copy()
    df = expand_calc_features_robust(df)
    
    if feature_name not in df.columns:
        print(f"  ERROR: Feature {feature_name} not found in expanded data")
        return pd.DataFrame()
    
    # Separate train and test
    train_mask = df['Data_selection'] == 'Train'
    test_mask = df['Data_selection'] != 'Train'
    
    train_data = df[train_mask][feature_name].dropna()
    test_data = df[test_mask][feature_name].dropna()
    
    print(f"\n  Data Summary:")
    print(f"    Training samples: {len(train_data):,}")
    print(f"    Test samples: {len(test_data):,}")
    print(f"    Training min: {train_data.min():.4f}")
    print(f"    Training max: {train_data.max():.4f}")
    print(f"    Test min: {test_data.min():.4f}")
    print(f"    Test max: {test_data.max():.4f}")
    
    # Identify feature type
    feature_types = identify_feature_types(df, [feature_name])
    
    if feature_name in feature_types['numerical']:
        print(f"  Feature Type: Numerical")
        # Analyze decile bins
        bin_info = analyze_decile_bins(df, feature_name, train_mask)
        
        if bin_info:
            print(f"  Decile Bins Analysis:")
            print(f"    Number of bins: {len(bin_info['bins'])-1}")
            print(f"    Decile values: {[f'{x:.4f}' for x in bin_info['decile_values']]}")
            
            print(f"\n    Bin Statistics:")
            for stat in bin_info['bin_stats']:
                print(f"      Bin {stat['bin']}: [{stat['lower_bound']:.4f}, {stat['upper_bound']:.4f}) - "
                      f"{stat['count']} samples ({stat['percentage']:.1f}%)")
        
        # Apply binning to test data
        binning_info = create_bins_for_features(
            df, 
            [feature_name], 
            [], 
            df[train_mask]
        )
        
        if feature_name in binning_info:
            df['binned'] = apply_binning(df, feature_name, binning_info[feature_name])
            
            print(f"\n  Test Data Binning Results:")
            test_binned = df[test_mask]['binned'].value_counts().sort_index()
            for bin_name, count in test_binned.items():
                print(f"    {bin_name}: {count} samples")
            
            # Check for out-of-range values
            out_of_range = (df[test_mask]['binned'] == 'Out_of_Range').sum()
            if out_of_range > 0:
                print(f"\n  WARNING: {out_of_range} test samples fell outside training range!")
                
    else:
        print(f"  Feature Type: Categorical")
        # Analyze categorical matching
        matching_info = validate_categorical_matching(df, feature_name, train_mask, test_mask)
        
        print(f"  Categorical Matching Analysis:")
        print(f"    Training categories: {matching_info['train_categories_count']}")
        print(f"    Test categories: {matching_info['test_categories_count']}")
        print(f"    Common categories: {matching_info['common_categories_count']}")
        print(f"    Categories only in test: {matching_info['categories_only_in_test_count']}")
        
        if matching_info['categories_only_in_test_count'] > 0:
            print(f"\n    Categories only in test data:")
            for cat in matching_info['categories_only_in_test'][:10]:  # Show first 10
                print(f"      - {cat}")
            if len(matching_info['categories_only_in_test']) > 10:
                print(f"      ... and {len(matching_info['categories_only_in_test']) - 10} more")
    
    return pd.DataFrame([{
        'model': model_display_name,
        'version': model_version_id,
        'feature': feature_name,
        'train_samples': len(train_data),
        'test_samples': len(test_data)
    }])


# ============================================================================
# VALIDATION FUNCTION (UNCHANGED)
# ============================================================================

def validate_psi_counts(psi_results: pd.DataFrame, dfcombined: pd.DataFrame) -> pd.DataFrame:
    """
    Validate the counts in PSI results against the original data.
    """
    validation_results = []
    
    for idx, row in psi_results.iterrows():
        model_display_name = row['modelDisplayName']
        model_version_id = row['modelVersionId']
        config_trench = row['trenchCategory']  # This is from config: can be 'ALL' or specific
        month = row['Month']
        segment_col = row['Segment_Column']
        segment_val = row['Segment_Value']
        
        try:
            # Build base filters for model
            base_filters = [
                f"modelDisplayName == '{model_display_name}'",
                f"modelVersionId == '{model_version_id}'"
            ]
            
            # Handle trenchCategory from config
            if config_trench != 'ALL':
                # If config has specific trench, filter by it
                base_filters.append(f"trenchCategory == '{config_trench}'")
            # If config_trench is 'ALL', we don't filter by trenchCategory
            
            # For segment-specific validation
            if segment_col != 'Overall':
                # Add segment filter
                base_filters.append(f"{segment_col} == '{segment_val}'")
            
            # TRAINING DATA COUNT
            # Training data is all data with Data_selection = 'Train'
            train_filters = base_filters + ["Data_selection == 'Train'"]
            
            # Build query for training data
            train_query = ' & '.join(train_filters)
            train_data = dfcombined.query(train_query)
            actual_train_count = train_data['digitalLoanAccountId'].nunique()
            
            # TEST DATA COUNT
            # Test data is for specific month and Data_selection != 'Train'
            test_filters = base_filters + [
                f"Application_month == '{month}'",
                "Data_selection != 'Train'"
            ]
            
            # Build query for test data
            test_query = ' & '.join(test_filters)
            test_data = dfcombined.query(test_query)
            actual_test_count = test_data['digitalLoanAccountId'].nunique()
            
            # Compare with PSI results
            validation = {
                'Row_Index': idx,
                'Model': model_display_name,
                'ModelVersion': model_version_id,
                'Config_Trench': config_trench,
                'Month': month,
                'Segment_Column': segment_col,
                'Segment_Value': segment_val,
                'PSI_Base_Count': row['Base_Count'],
                'Actual_Train_Count': actual_train_count,
                'PSI_Actual_Count': row['Actual_Count'],
                'Actual_Test_Count': actual_test_count,
                'Train_Match': row['Base_Count'] == actual_train_count,
                'Test_Match': row['Actual_Count'] == actual_test_count,
                'Train_Difference': row['Base_Count'] - actual_train_count,
                'Test_Difference': row['Actual_Count'] - actual_test_count
            }
            
            validation_results.append(validation)
            
        except Exception as e:
            print(f"Error validating row {idx}: {e}")
            validation = {
                'Row_Index': idx,
                'Model': model_display_name,
                'ModelVersion': model_version_id,
                'Config_Trench': config_trench,
                'Month': month,
                'Segment_Column': segment_col,
                'Segment_Value': segment_val,
                'PSI_Base_Count': row['Base_Count'],
                'Actual_Train_Count': 'ERROR',
                'PSI_Actual_Count': row['Actual_Count'],
                'Actual_Test_Count': 'ERROR',
                'Train_Match': False,
                'Test_Match': False,
                'Train_Difference': 'ERROR',
                'Test_Difference': 'ERROR',
                'Error': str(e)
            }
            validation_results.append(validation)
    
    return pd.DataFrame(validation_results)


# # ============================================================================
# # MAIN EXECUTION (OPTIONAL - FOR TESTING)
# # ============================================================================

# def main():
#     """
#     Example usage with decile binning.
#     """
#     # Load your data
#     # dfcombined = pd.read_csv('your_data.csv')
#     # configdf = pd.read_csv('your_config.csv')
    
#     # Example data structure
#     dfcombined = pd.DataFrame()  # Your actual dataframe
#     configdf = pd.DataFrame()    # Your actual config
    
#     # Calculate PSI with decile binning
#     print("="*80)
#     print("CALCULATING PSI WITH DECILE BINNING")
#     print("="*80)
    
#     psi_results = calculate_psi_for_model(
#         dfcombined=dfcombined,
#         configdf=configdf,
#         model_display_name='cic_model_sil',
#         debug=True
#     )
    
#     if not psi_results.empty:
#         # Save results
#         psi_results.to_csv('psi_results_decile_binning.csv', index=False)
#         print(f"\nResults saved to psi_results_decile_binning.csv")
        
#         # Summary of decile binning
#         print("\n" + "="*80)
#         print("DECILE BINNING SUMMARY")
#         print("="*80)
        
#         numerical_features = psi_results[psi_results['Feature_Type'] == 'numerical']['Feature'].unique()
#         categorical_features = psi_results[psi_results['Feature_Type'] == 'categorical']['Feature'].unique()
        
#         print(f"Numerical features with decile binning: {len(numerical_features)}")
#         print(f"Categorical features with exact matching: {len(categorical_features)}")
        
#         # Show features with highest PSI
#         high_psi = psi_results[psi_results['PSI'] > 0.2]
#         if len(high_psi) > 0:
#             print(f"\nFeatures with PSI > 0.2 (moderate or significant shift):")
#             top_features = high_psi.groupby(['Feature', 'Feature_Type'])['PSI'].max().sort_values(ascending=False).head(10)
#             for (feature, ftype), psi_val in top_features.items():
#                 print(f"  {feature} ({ftype}): {psi_val:.4f}")
    
#     return psi_results


# if __name__ == "__main__":
#     # Run the analysis with decile binning
#     results = main()

# Config query

In [7]:
sq = """ WITH base AS (
  -- First part with actual trenchCategory
  SELECT 
    modelDisplayName, 
    modelVersionId, 
    CASE 
      WHEN trenchCategory IS NULL THEN 'ALL'
      WHEN trenchCategory = '' THEN 'ALL'
      ELSE trenchCategory 
    END AS trenchCategory,
(CASE 
      WHEN modelDisplayName LIKE '%sil%' THEN 'SIL'
      WHEN modelDisplayName LIKE '%cash%' THEN 'CASH'
      ELSE 'ALL' 
    END) AS product_category,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`

  
  UNION ALL
  
  -- Second part with 'ALL' trenchCategory
  SELECT 
    modelDisplayName, 
    modelVersionId, 
    'ALL' AS trenchCategory,
    (CASE 
      WHEN modelDisplayName LIKE '%sil%' THEN 'SIL'
      WHEN modelDisplayName LIKE '%cash%' THEN 'CASH'
      ELSE 'ALL' 
    END) AS product_category,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE trenchCategory IS NOT NULL

)
SELECT distinct
  modelDisplayName, 
  modelVersionId, 
  trenchCategory, 
  product_category,
FROM base
ORDER BY 4, 1, 2, 3;"""

configdf = client.query(sq).to_dataframe()
configdf

,modelDisplayName,modelVersionId,trenchCategory,product_category
0,alpha_income_model,v1,ALL,ALL
1,beta_income_model,v1,ALL,ALL
2,alpha_stack_model_cash,v1,ALL,CASH
3,alpha_stack_model_cash,v1,Trench 1,CASH
4,alpha_stack_model_cash,v1,Trench 2,CASH
5,alpha_stack_model_cash,v1,Trench 3,CASH
6,alpha_stack_model_cash,v1.1,ALL,CASH
7,alpha_stack_model_cash,v1.1,Trench 1,CASH
8,alpha_stack_model_cash,v1.1,Trench 2,CASH
9,apps_score_cash,v1,ALL,CASH


In [8]:
configdf.to_csv('configdf.csv', index = False)
configdf.to_pickle('configdf.pkl')

## SIL

### Queries

### cic_model_sil

### Test

In [9]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details

sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
        case when modelDisplayName = 'Alpha - CIC-SIL-Model' then 'cic_model_sil' else modelDisplayName end as modelDisplayName    
    ,modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and prediction is not null
  ),
base as
(SELECT distinct

--Alpha_cic_sil_score
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'cic_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
)
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

The shape of the dataframe is: (107557, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,2402440,b0e95c66-5ce7-4ece-aeff-4d3b28dad01c,0.11136085020144332,2025-11-07 09:10:35.213600,2025-11-07 09:10:35.219746,cic_model_sil,v1,SIL Competitor,F,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1762473600000,""cic_Personal_Loans_...",Test,2025-11-07 17:10:23,2025-11-07 17:40:00,2025-11
1,2539836,6c6703c4-5f24-4d57-b983-d63b977f3619,0.14965652480021718,2025-05-14 05:50:15.640022,2025-05-14 05:50:15.646151,cic_model_sil,v1,SIL ZERO,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1747180800000,""cic_Personal_Loans_...",Test,2025-05-14 13:50:07,NaT,2025-05
2,2610988,3cd006dd-1f7c-473b-8459-b2ec2dac4720,0.1619340616351965,2025-09-09 09:23:58.214000,2025-09-09 09:23:58.220394,cic_model_sil,v1,SIL Competitor,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1757376000000,""cic_Personal_Loans_...",Test,2025-09-09 17:23:49,NaT,2025-09
3,2684028,24f1f58f-6262-480c-8db2-31177064bd0e,0.17056241682938522,2025-10-05 04:44:08.379022,2025-10-05 04:44:08.385506,cic_model_sil,v1,SIL-Instore,F,Mall,android,cic_model_sil,SIL,ALL,"{""run_date"":1759622400000,""cic_Personal_Loans_...",Test,2025-10-05 12:43:57,2025-10-10 15:05:19,2025-10
4,2698089,5756ec8e-eb89-4b01-a595-b8feb6f5fe2f,0.11963830395919038,2025-07-27 10:32:02.703559,2025-07-27 10:32:02.709721,cic_model_sil,v1,SIL-Instore,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1753574400000,""cic_Personal_Loans_...",Test,2025-07-27 18:31:54,2025-07-27 18:35:26,2025-07


In [10]:
df1 = dfd.copy()

### Train

In [11]:
sq = """WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    
    case when modelDisplayName = 'Alpha - CIC-SIL-Model' then 'cic_model_sil' else modelDisplayName end as modelDisplayName
    
    ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
base as 
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (450861, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1074411,90da4ddc-c0eb-4b46-9a39-a9e872a0cac6,0.225131,2025-12-13T11:41:46.108896,2025-12-13T11:41:46.108896,cic_model_sil,v2,SIL-Instore,F,Mall,android,Alpha - CIC-SIL-Model,SIL,Trench 2,"{""ScoreRange"": ""Bi"", ""ln_loan_level_user_type""...",Train,2024-12-29 15:47:35,NaT,2024-12
1,1779527,e5578538-61b5-44e4-bedd-94824be363d6,0.360614,2025-12-13T11:41:46.899618,2025-12-13T11:41:46.899618,cic_model_sil,v2,SIL-Instore,M,Appliance,ios,Alpha - CIC-SIL-Model,SIL,Trench 2,"{""ScoreRange"": ""NH_Hi"", ""ln_loan_level_user_ty...",Train,2025-08-21 15:15:40,2025-08-23 16:44:14,2025-08
2,1963733,cb444a6c-1165-4f7a-a393-72b576ab7a7c,0.156348,2025-12-13T11:19:39.915690,2025-12-13T11:19:39.915690,cic_model_sil,v1,SIL-Instore,M,Appliance,ios,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_days_since_last_inquiry"": 330.0, ""cic_ve...",Train,2023-03-27 19:55:24,2023-03-27 20:04:58,2023-03
3,1967303,3ea0aa8c-f9b6-4898-9cb4-4d75410811da,0.143880,2025-12-13T11:19:12.930842,2025-12-13T11:19:12.930842,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_days_since_last_inquiry"": 61.0, ""cic_vel...",Train,2023-03-29 18:54:45,2023-03-29 18:58:01,2023-03
4,1994673,a58514f4-dacd-4ded-9744-776d00aa0904,0.124429,2025-12-13T11:19:15.036968,2025-12-13T11:19:15.036968,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2023-04-14 13:47:08,2023-04-14 17:34:58,2023-04


In [12]:
df2 = dfd.copy()

In [13]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (558418, 19)


In [14]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (558418, 19)
The shape of the dataframe after dropping duplicates is: (558418, 19)


In [15]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [30]:
df_combined.sample(100).to_csv('sample_data.csv', index = False)

### PSI calculation

In [16]:
# Calculate PSI for a specific model
psi_results = calculate_psi_for_model(
    dfcombined=df_combined,
    configdf=configdf,
    model_display_name='cic_model_sil',  # Your model name
    debug=False  # Set to False for production
)


Starting PSI Pipeline for Model: cic_model_sil
Total combinations to process: 5

Processing combination 56/5: modelVersionId=v1, trenchCategory=ALL
  Data points: 307884
  Features identified: 11
Identified 9 numerical and 2 categorical features
  Creating decile bins from training data...
    Created 9 decile bins for v1_Calc_cic_days_since_last_inquiry (min=0.0000, max=11649.0000)
    Created 3 decile bins for v1_Calc_cic_cnt_active_contracts (min=1.0000, max=101.0000)
    Created 10 decile bins for v1_Calc_cic_max_amt_granted_24M (min=0.0000, max=8000000.0000)
    Created 10 decile bins for v1_Calc_cic_tot_active_contracts_util (min=-1.0000, max=352.8822)
    Created 5 decile bins for v1_Calc_cic_vel_contract_granted_amt_12on24 (min=0.0000, max=29.2579)
    Created 10 decile bins for v1_Calc_cic_Personal_Loans_granted_contracts_amt_24M (min=0.0000, max=6042680.0000)
    Created 4 decile bins for v1_Calc_cic_vel_contract_nongranted_cnt_12on24 (min=0.1990, max=2.0120)
    Created 10 

In [17]:
psi_results.head()

,modelDisplayName,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,Base_Count,Actual_Count,Expected_Percentage,Actual_Percentage,PSI,PSI_Interpretation
100,cic_model_sil,v1,ALL,score,numerical,Overall,All,2025-03,All_Training_Months,2025-03,204453,180,10.0,10.0,0.149367,Stable
727,cic_model_sil,v1,ALL,score,numerical,loan_product_type,Appliance,2025-03,All_Training_Months,2025-03,163711,158,10.0,10.0,0.135054,Stable
837,cic_model_sil,v1,ALL,score,numerical,loan_product_type,Mall,2025-03,All_Training_Months,2025-03,9507,22,10.0,12.5,2.207262,Significant Shift
210,cic_model_sil,v1,ALL,score,numerical,new_loan_type,SIL-Instore,2025-03,All_Training_Months,2025-03,185552,176,10.0,10.0,0.154039,Stable
320,cic_model_sil,v1,ALL,score,numerical,new_loan_type,SIL ZERO,2025-03,All_Training_Months,2025-03,10397,4,10.0,50.0,17.809518,Significant Shift


In [18]:
psi_results['Base_Month'].unique()

array(['All_Training_Months'], dtype=object)

In [19]:
# After calculating PSI, validate the counts
validation_results = validate_psi_counts(psi_results, df_combined)

# Check for mismatches
mismatches = validation_results[
    (validation_results['Train_Match'] == False) | 
    (validation_results['Test_Match'] == False)
]

if len(mismatches) > 0:
    print("Mismatches found:")
    print(mismatches)
else:
    print("All counts match!")

All counts match!


In [20]:
import re

psi_results['Feature'] = psi_results['Feature'].str.replace('_Calc_', '_', regex=False)

In [21]:
psi_results['Feature'].value_counts(dropna=False)

Feature
score                                              141
v1_cic_vel_contract_granted_amt_12on24              77
v1_cic_Personal_Loans_granted_contracts_amt_24M     77
v1_cic_zero_non_granted_ever_flag                   77
v1_cic_zero_granted_ever_flag                       77
v1_cic_vel_contract_nongranted_cnt_12on24           77
v1_run_date                                         77
v1_cic_tot_active_contracts_util                    77
v1_cic_max_amt_granted_24M                          77
v1_cic_days_since_last_inquiry                      77
v1_cic_cnt_active_contracts                         77
v2_flg_zero_non_granted_ever                        64
v2_vel_contract_granted_amt_6on12                   64
v2_vel_contract_closed_amt_3on12                    64
v2_total_overdue_granted_contracts                  64
v2_tot_active_contracts_util                        64
v2_max_amt_granted_24M                              64
v2_ln_loan_level_user_type                          64
v2

In [22]:

# List of features to remove
remove_features = ['v2_customerId', 'v2_digitalLoanAccountId','v2_crifApplicationId', 'v1_run_date']

# Drop rows where feature is in the list
psi_results = psi_results[~psi_results['Feature'].isin(remove_features)]

# Replace 'score' with 'Alpha_cic_sil_score' in the Feature column
psi_results['Feature'] = psi_results['Feature'].replace('score', 'Alpha_cic_sil_score')

# Replace values starting with 'calc_' by removing the prefix
# psi_results['Feature'] = psi_results['Feature'].apply(
#     lambda x: x[5:] if x.startswith('calc_') else x
# )



In [23]:
psi_results[['modelVersionId','Feature']].value_counts()

modelVersionId  Feature                                        
v1              Alpha_cic_sil_score                                77
                v1_cic_vel_contract_granted_amt_12on24             77
                v1_cic_Personal_Loans_granted_contracts_amt_24M    77
                v1_cic_zero_granted_ever_flag                      77
                v1_cic_vel_contract_nongranted_cnt_12on24          77
                v1_cic_zero_non_granted_ever_flag                  77
                v1_cic_tot_active_contracts_util                   77
                v1_cic_max_amt_granted_24M                         77
                v1_cic_days_since_last_inquiry                     77
                v1_cic_cnt_active_contracts                        77
v2              v2_granted_contracts_cnt_6M                        64
                v2_vel_contract_granted_amt_6on12                  64
                v2_vel_contract_closed_amt_3on12                   64
                v2_total_o

In [24]:
psi_results[psi_results['Feature'] == 'Alpha_cic_sil_score']

,modelDisplayName,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,Base_Count,Actual_Count,Expected_Percentage,Actual_Percentage,PSI,PSI_Interpretation
100,cic_model_sil,v1,ALL,Alpha_cic_sil_score,numerical,Overall,All,2025-03,All_Training_Months,2025-03,204453,180,10.0,10.000000,0.149367,Stable
727,cic_model_sil,v1,ALL,Alpha_cic_sil_score,numerical,loan_product_type,Appliance,2025-03,All_Training_Months,2025-03,163711,158,10.0,10.000000,0.135054,Stable
837,cic_model_sil,v1,ALL,Alpha_cic_sil_score,numerical,loan_product_type,Mall,2025-03,All_Training_Months,2025-03,9507,22,10.0,12.500000,2.207262,Significant Shift
210,cic_model_sil,v1,ALL,Alpha_cic_sil_score,numerical,new_loan_type,SIL-Instore,2025-03,All_Training_Months,2025-03,185552,176,10.0,10.000000,0.154039,Stable
320,cic_model_sil,v1,ALL,Alpha_cic_sil_score,numerical,new_loan_type,SIL ZERO,2025-03,All_Training_Months,2025-03,10397,4,10.0,50.000000,17.809518,Significant Shift
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,cic_model_sil,v2,Trench 3,Alpha_cic_sil_score,numerical,new_loan_type,SIL-Instore,2025-12,All_Training_Months,2025-12,8439,90,10.0,10.000000,0.249805,Moderate Shift
1926,cic_model_sil,v2,Trench 3,Alpha_cic_sil_score,numerical,new_loan_type,SIL Competitor,2025-12,All_Training_Months,2025-12,2136,42,10.0,10.000000,0.425207,Moderate Shift
1966,cic_model_sil,v2,Trench 3,Alpha_cic_sil_score,numerical,new_loan_type,SIL ZERO,2025-12,All_Training_Months,2025-12,940,9,10.0,20.000000,10.525592,Significant Shift
2006,cic_model_sil,v2,Trench 3,Alpha_cic_sil_score,numerical,osType,android,2025-12,All_Training_Months,2025-12,10318,130,10.0,10.000000,0.148006,Stable


In [26]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v6"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=91b1d8b6-99d0-4e64-abe0-eedc27df48d6>

### Alpha Sil Stack Model

### Test

In [ ]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details
# Sil_Alpha_Stack_score
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
        case when modelDisplayName = 'Alpha - StackingModel' then 'alpha_stack_model_sil' else modelDisplayName end as modelDisplayName    
    ,modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
base as
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'alpha_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
)
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    
    case when modelDisplayName = 'Alpha - StackingModel' then 'alpha_stack_model_sil' else modelDisplayName end as modelDisplayName 
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
base as 
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'alpha_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

In [ ]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [ ]:
# Calculate PSI for a specific model
psi_results = calculate_psi_for_model(
    dfcombined=df_combined,
    configdf=configdf,
    model_display_name='alpha_stack_model_sil',  # Your model name
    debug=False  # Set to False for production
)
psi_results.head()

In [ ]:
psi_results['Feature'] = psi_results['Feature'].str.replace('_Calc_', '_', regex=False)
psi_results['Feature'].value_counts(dropna=False)


In [ ]:
psi_results['Feature'].unique()

In [ ]:

# List of features to remove, '
remove_features = [ 'v2_trench_category',
       'v2_ln_os_type', 'v2_is_android', 'v2_sb_demo_score_norm',
       'v2_apps_score_norm', 'v2_s_credo_score_norm',
       'v2_sa_cic_score_norm',
                       ]
# Drop rows where feature is in the list
psi_results = psi_results[~psi_results['Feature'].isin(remove_features)]

# Replace 'score' with 'Sil_Alpha_Stack_score' in the Feature column
psi_results['Feature'] = psi_results['Feature'].replace('score', 'Sil_Alpha_Stack_score')

# # Replace values starting with 'calc_' by removing the prefix
# psi_results['Feature'] = psi_results['Feature'].apply(
#     lambda x: x[5:] if x.startswith('calc_') else x
# )




In [ ]:
psi_results[['modelVersionId','Feature']].value_counts()
# df2.rename(columns={'calc_beta_demo_score':'calc_sb_demo_score',
#                      'calc_cic_score':'calc_s_cic_score',
#                       'calc_apps_score':'calc_s_apps_score',
#                         'calc_credo_gen_score':'calc_s_credo_score'}, inplace = True)

In [ ]:
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi_v6"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

### Beta Sil App Score

### Test

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
    
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'apps_score_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_app_score
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base where lower(new_loan_type) like '%sil%'
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil' else modelDisplayName end as modelDisplayName 
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'apps_score_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base where lower(new_loan_type) like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe dfd is:\{dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

In [ ]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [ ]:
# Usage Example:
# Calculate PSI for a specific model
psi_results = calculate_psi_for_model(
    dfcombined=df_combined,
    configdf=configdf,
    model_display_name='apps_score_model_sil',  # Your model name
    debug=False  # Set to False for production
)
psi_results.head()

In [ ]:

psi_results['Feature'] = psi_results['Feature'].str.replace('_Calc_', '_', regex=False)
psi_results[['modelVersionId','Feature']].value_counts()


In [ ]:

# List of features to remove, '
remove_features = ['v2_appScoreModel'                       ]
# Drop rows where feature is in the list
psi_results = psi_results[~psi_results['Feature'].isin(remove_features)]

# Replace 'score' with 'Sil_Alpha_Stack_score' in the Feature column
psi_results['Feature'] = psi_results['Feature'].replace('score', 'sil_beta_app_score')

# # Replace values starting with 'calc_' by removing the prefix
# psi_results['Feature'] = psi_results['Feature'].apply(
#     lambda x: x[5:] if x.startswith('calc_') else x
# )




In [ ]:
psi_results[['modelVersionId','Feature']].value_counts()

In [ ]:
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi_v6"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

### Beta SIL Demo Score

### Test

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_demo_score
  prediction as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base where lower(new_loan_type) like '%sil%'
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe for test is :\{dfd.shape}")
dfd.head()

In [ ]:
dd.query("""select digitalLoanAccountId, count(digitalLoanAccountId)cnt from dfd group by digitalLoanAccountId having count(digitalLoanAccountId)> 1 order by 2 desc""").to_df()

In [ ]:
dfd[dfd['digitalLoanAccountId'] == 'c00e08a9-b103-4904-b3c2-b0be39be5af2']

In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName    
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  prediction as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base where lower(new_loan_type) like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe for train is :\{dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

In [ ]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [ ]:
# Usage Example:
# Calculate PSI for a specific model
psi_results = calculate_psi_for_model(
    dfcombined=df_combined,
    configdf=configdf,
    model_display_name='beta_demo_model_sil',  # Your model name
    debug=False  # Set to False for production
)
psi_results.head()

In [ ]:
# After calculating PSI, validate the counts
validation_results = validate_psi_counts(psi_results, df_combined)

# Check for mismatches
mismatches = validation_results[
    (validation_results['Train_Match'] == False) | 
    (validation_results['Test_Match'] == False)
]

if len(mismatches) > 0:
    print("Mismatches found:")
    print(mismatches)
else:
    print("All counts match!")

In [ ]:

psi_results['Feature'] = psi_results['Feature'].str.replace('_Calc_', '_', regex=False)
psi_results[['modelVersionId','Feature']].value_counts()


In [ ]:

# List of features to remove, '
# remove_features = ['v2_appScoreModel'                       ]
# Drop rows where feature is in the list
psi_results = psi_results[~psi_results['Feature'].isin(remove_features)]

# Replace 'score' with 'Sil_Alpha_Stack_score' in the Feature column
psi_results['Feature'] = psi_results['Feature'].replace('score', 'sil_beta_demo_score')

# # Replace values starting with 'calc_' by removing the prefix
# psi_results['Feature'] = psi_results['Feature'].apply(
#     lambda x: x[5:] if x.startswith('calc_') else x
# )




In [ ]:
psi_results[['modelVersionId','Feature']].value_counts()

In [ ]:
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi_v6"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

## Beta SIL STACK Score Model

### Test

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = ''Beta - StackScoreModel' then 'beta_stack_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_stack_score
  prediction as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName    
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  prediction as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
df2 = dfd.copy()

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

In [ ]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [ ]:
# Usage Example:
psi_results = calculate_psi_for_model(
    dfcombined=df_combined,
    configdf=configdf,
    model_display_name='beta_demo_model_sil'
)
psi_results.head()

In [ ]:

psi_results['Feature'] = psi_results['Feature'].str.replace('_Calc_', '_', regex=False)
psi_results[['modelVersionId','Feature']].value_counts()


In [ ]:

# List of features to remove, '
remove_features = ['v2_appScoreModel'                       ]
# Drop rows where feature is in the list
psi_results = psi_results[~psi_results['Feature'].isin(remove_features)]

# Replace 'score' with 'Sil_Alpha_Stack_score' in the Feature column
psi_results['Feature'] = psi_results['Feature'].replace('score', 'sil_beta_demo_score')

# Replace values starting with 'calc_' by removing the prefix
psi_results['Feature'] = psi_results['Feature'].apply(
    lambda x: x[5:] if x.startswith('calc_') else x
)




In [ ]:
psi_results[['modelVersionId','Feature']].value_counts()

In [ ]:
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

# End